In [26]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import numpy as np
import os
import time


path_to_file = 'TrainingData-Copy1.txt'
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
vocab = sorted(set(text))

def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

ids_from_chars = preprocessing.StringLookup(
    vocabulary=list(vocab), mask_token=None)

all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)


def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)


BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

#dataset


class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(rnn_units,
                                       return_sequences=True,
                                      return_state = True)
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = tf.keras.layers.GRU(rnn_units,
                                       return_sequences=False,
                                      return_state = True)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x

model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)


loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss=loss)

# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

EPOCHS = 30
history = model.fit(dataset, epochs=EPOCHS)

Epoch 1/30


TypeError: in user code:

    C:\Users\William\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:855 train_function  *
        return step_function(self, iterator)
    <ipython-input-26-6135447e178e>:65 call  *
        x = self.dense(x, training=training)
    C:\Users\William\anaconda3\lib\site-packages\tensorflow\python\keras\engine\base_layer.py:1013 __call__  **
        input_spec.assert_input_compatibility(self.input_spec, inputs, self.name)
    C:\Users\William\anaconda3\lib\site-packages\tensorflow\python\keras\engine\input_spec.py:197 assert_input_compatibility
        raise TypeError('Inputs to a layer should be tensors. Got: %s' % (x,))

    TypeError: Inputs to a layer should be tensors. Got: <tensorflow.python.keras.layers.recurrent_v2.GRU object at 0x000001EBD512D100>


In [6]:
print(model.summary())

Model: "my_model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      multiple                  24832     
_________________________________________________________________
lstm_1 (LSTM)                multiple                  18882560  
_________________________________________________________________
dense_1 (Dense)              multiple                  198753    
Total params: 19,106,145
Trainable params: 19,106,145
Non-trainable params: 0
_________________________________________________________________
None


In [18]:
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=0.45):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states1, states2 = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states1, states2

one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [20]:
start = time.time()
states = None
next_char = tf.constant(['Introduction:'])
result = [next_char]

for n in range(1000):
  next_char, states1, states2 = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ValueError: in user code:

    <ipython-input-18-64e726a50e7c>:30 generate_one_step  *
        predicted_logits, states1, states2 = self.model(inputs=input_ids, states=states,

    ValueError: not enough values to unpack (expected 3, got 2)


In [5]:
EPOCHS = 30
history = model.fit(dataset, epochs=EPOCHS)

Epoch 1/30
9/9 [==============================] - 5s 546ms/step - loss: 1.9277
Epoch 2/30
9/9 [==============================] - 5s 548ms/step - loss: 1.8973
Epoch 3/30
9/9 [==============================] - 5s 549ms/step - loss: 1.8706
Epoch 4/30
9/9 [==============================] - 5s 548ms/step - loss: 1.8412
Epoch 5/30
9/9 [==============================] - 5s 550ms/step - loss: 1.8161
Epoch 6/30
9/9 [==============================] - 5s 549ms/step - loss: 1.7861
Epoch 7/30
9/9 [==============================] - 5s 549ms/step - loss: 1.7561
Epoch 8/30
9/9 [==============================] - 5s 550ms/step - loss: 1.7311
Epoch 9/30
9/9 [==============================] - 5s 550ms/step - loss: 1.7030
Epoch 10/30
9/9 [==============================] - 5s 551ms/step - loss: 1.6801
Epoch 11/30
9/9 [==============================] - 5s 551ms/step - loss: 1.6510
Epoch 12/30
9/9 [==============================] - 5s 552ms/step - loss: 1.6283
Epoch 13/30
9/9 [==============================] 

In [9]:
start = time.time()
states = None
next_char = tf.constant(['Introduction'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Introduction cke onalars ug on panthint isthesithaly Is tio pe So inge the t teme hon f bes ive ant pre d thepl Ne Amens, th conanen’s prato hes hemowowher of pathe t d ole s s he orohe te ame wo s con’sinathewithe an thave ald cte (Fril t t f calico the witored wers hers ciesthol she thesthe theded
f UF t t th incerin panthe caneretonenthe anancthe f taden ane thes; crt mer cof ald Untheld a h thiontson f Catr in
s. Scangadiongecalis se te h pan tor
bof ime iont Sivepe ant plericqurtheprs ofond antin athe on ares pal the thin trthe. thear theshe Nowor althe thed thiner thely to% Scan the s thinspr pon sene an th pe we pe poher pathe thar, acantetin’sth of fourers thethe tan f the ofan than tha ingel t he con thathen the.” s. ntes the paratemof thes aroucofers s tior
egrithors athe Man Scanis w
owhe there iore stherosh t than a calin urorinatracoush he pe clale WWTh ile orif alancalalanencont at fongin pathomod the he worous s ther antiven Itorstasheald hthertesinthe at
fithe Manand
he

In [42]:
tf.saved_model.save(one_step_model, '2_Layer_GRU')


FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.



FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.


INFO:tensorflow:Assets written to: 2_Layer_GRU\assets


INFO:tensorflow:Assets written to: 2_Layer_GRU\assets
